In [1]:
import duckdb
import pandas as pd

%load_ext sql

In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [3]:
%sql duckdb:///demo.db
# CHANGE ACCORDING TO YOUR DB FILE PATH

### Who are the top 10 users with respect to number of songs listened to?

In [4]:
%%sql
WITH top_ten_user_cte AS (
    SELECT user_id,
        COUNT(listen_id) AS listen_count
    from user_listens
    GROUP BY user_id
    ORDER BY listen_count DESC
    LIMIT 10
)
SELECT u.user_name
FROM top_ten_user_cte tt
    INNER JOIN users u ON tt.user_id = u.user_id;

,user_name
0,kingkool68
1,kmontenegro
2,Silent Singer
3,Cl�psHydra
4,6d6f7274686f6e
5,Grawl
6,budkin
7,Groschi
8,InvincibleAsia
9,drpfenderson


### How many users did listen to some song on the 1st of March 2019?

In [5]:
%%sql
WITH date_cte AS (
    SELECT listen_time
    FROM time
    WHERE year = 2019
        AND month = 3
        AND day = 1
)
SELECT COUNT(DISTINCT ul.user_id) AS user_count
FROM date_cte d
    INNER JOIN user_listens ul ON d.listen_time = ul.listen_time;

,user_count
0,62


### For every user, what was the first song the user listened to?

In [6]:
%%sql
WITH user_songs_cte AS(
    SELECT ul.song_name,
        ul.user_id,
        ROW_NUMBER() OVER(
            PARTITION BY ul.user_id
            ORDER BY ul.listen_time DESC
        ) num
    FROM user_listens ul
)
SELECT us.song_name,
    u.user_name
FROM user_songs_cte us
    INNER JOIN users u ON us.user_id = u.user_id
WHERE us.num = 1;

,song_name,user_name
0,Love You Like A Love Song,beefkidney
1,Let You Down,jin_tsu
2,Different Reality (Vertical Mode remix),zergut
3,100 Bad Days,Leonivek
4,Fleeting Love,meisterhauns
...,...,...
193,Latter Day,budkin
194,Sua Cara (feat. Anitta & Pabllo Vittar),marcianofonseca
195,Short Wave Lies (Live Version),anoubis
196,Ohlala (Radio Edit),phdnk


### For each user top 3 days on which they had the most listens and how many listens they had on each of these days.

In [7]:
%%sql
WITH user_listen_count_by_day_cte AS (
    SELECT DISTINCT ul.user_id as user,
        t.date as date,
        COUNT(ul.listen_id) OVER(
            PARTITION BY ul.user_id,
            t.date
            ORDER BY t.date DESC
        ) listen_count
    FROM user_listens ul
        INNER JOIN time t ON ul.listen_time = t.listen_time
    ORDER BY listen_count DESC
), 
song_rank_cte AS (
    SELECT user,
        date,
        listen_count,
        DENSE_RANK() OVER(
            PARTITION BY user
            ORDER BY listen_count DESC
        ) as listen_rank
    FROM user_listen_count_by_day_cte
)
SELECT user,
    listen_count AS number_of_listens,
    date
FROM song_rank_cte
WHERE listen_rank IN (1,2,3) ORDER BY user, number_of_listens DESC;

,user,number_of_listens,date
0,006908d6ef8d9ef1c68c8617e71486c3d5688ce295e034...,69,2019-02-15
1,006908d6ef8d9ef1c68c8617e71486c3d5688ce295e034...,46,2019-02-18
2,006908d6ef8d9ef1c68c8617e71486c3d5688ce295e034...,30,2019-02-19
3,006908d6ef8d9ef1c68c8617e71486c3d5688ce295e034...,30,2019-02-14
4,00b329e16d61f87008dd427f3816a41174a77234223005...,100,2019-03-22
...,...,...,...
611,fb56d69c9cfee8fb7e2c7956afc640414d1ada9501cef0...,52,2019-02-11
612,fb56d69c9cfee8fb7e2c7956afc640414d1ada9501cef0...,51,2019-02-14
613,fbeb32aa10940244c96350da9e8650bbf29d00f357b53b...,112,2019-03-21
614,fbeb32aa10940244c96350da9e8650bbf29d00f357b53b...,69,2019-04-14


### Calculate the absolute number of active users on daily basis and the percentage of active users among all users.

In [8]:
%%sql 
WITH lag_cte AS(
    SELECT ul.user_id,
        t.date as active_date,
        LAG(t.date, 6) OVER(
            PARTITION BY ul.user_id
            ORDER BY t.date
        ) six_day_previous
    FROM user_listens ul
        INNER JOIN time t ON ul.listen_time = t.listen_time
),
active_users_cte AS (
    SELECT DISTINCT user_id,
        active_date
    FROM lag_cte
    WHERE six_day_previous IS DISTINCT
    FROM NULL
        AND DATEDIFF('day', six_day_previous, active_date) = 6
    ORDER BY active_date
)
SELECT active_date,
    COUNT(user_id) OVER(PARTITION BY active_date) number_of_active_users,
    (
        number_of_active_users / (
            SELECT COUNT(DISTINCT user_id)
            FROM users
        ) * 100
    ) percentage_active_users
FROM active_users_cte
ORDER BY active_date;

,active_date,number_of_active_users,percentage_active_users
0,2019-01-07,1,0.495050
1,2019-01-08,1,0.495050
2,2019-01-09,2,0.990099
3,2019-01-09,2,0.990099
4,2019-01-10,2,0.990099
...,...,...,...
165,2019-04-12,2,0.990099
166,2019-04-13,4,1.980198
167,2019-04-13,4,1.980198
168,2019-04-13,4,1.980198
